In [ ]:
from PIL import Image
import os
import random

# Function to overlay images with transformations
def overlay_images_with_transformations(part_path, gripper_path, output_path):
    
    # Transformations to apply
    transformations = [
        {'type': 'rotate', 'value': random.randint(0, 360)},
        {'type': 'shift', 'x': random.randint(-50, 50), 'y': random.randint(-50, 50)}
    ]

    # Load images
    part = Image.open(part_path).convert("RGBA")
    gripper = Image.open(gripper_path).convert("RGBA")

    # Note: We don't resize the images now, but during training we will resize the images to a fixed size
    
    # Apply rotation transformation
    for transform in transformations:
        if transform['type'] == 'rotate':
            gripper = gripper.rotate(transform['value'], expand=True)
    
    # Calculate the center position for the gripper
    # center_x = (part.width - gripper.width) // 100
    # center_y = (part.height - gripper.height) // 100
    center_x = 0
    center_y = -98
    # print("Center x: ", center_x, "Center y: ", center_y)
    
    # Calculate the maximum allowable shift
    max_shift_x = (part.width - gripper.width) // 2
    max_shift_y = (part.height - gripper.height) // 2
    # print("Max shift x: ", max_shift_x, "Max shift y: ", max_shift_y)
    
    # Apply shift transformation
    for transform in transformations:
        if transform['type'] == 'shift':
            shift_x = center_x + min(max(transform['x'], -max_shift_x), max_shift_x) 
            shift_y = center_y + min(max(transform['y'], -max_shift_y), max_shift_y) 
            gripper = gripper.transform(part.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
            print("Shift x: ", shift_x, "Shift y: ", shift_y)
    
    # Overlay images
    combined = Image.alpha_composite(part, gripper)
    combined.save(output_path)

In [ ]:
# Paths to your images
gripper_file = "data/train_images/Grippers/Gripper_1.png"
part_folder = "data/train_images/Metal_sheets/"
output_folder = "data/train_images/Train/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [ ]:
import os

# Function to parse through a directory and apply a specified function to each .png file
def parse_directory_and_apply_function(root_directory):
    for subdir, _, files in os.walk(root_directory):
        for file in files:
            if file.endswith(".png"):
                # combine folder name and part name
                part = part_folder + file
                output_file = output_folder + file


                overlay_images_with_transformations(part, gripper_file, output_file)

parse_directory_and_apply_function(part_folder)